## Task 1 Solution

In [5]:
# Task 1: SqueezeNet for Dogs vs. Cats Dataset
# Import necessary libraries
!pip install mxnet==1.5.1
import mxnet as mx
import os
import zipfile
import time
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

# Download and prepare the Dogs vs. Cats dataset
!pip install opendatasets
import opendatasets as od
dataset_url = 'https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data'
od.download(dataset_url)
zip_file_path = '/content/dogs-vs-cats-redux-kernels-edition/train.zip'
extracted_dir = '/content/dogs-vs-cats-redux-kernels-edition/'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Verify the number of images (should be 12500 for each class)
mypath = os.path.join(os.getcwd(), '/content/dogs-vs-cats-redux-kernels-edition/train/')
cats_imgs = [os.path.join(mypath, f) for f in os.listdir(mypath) if f.startswith('cat')]
dogs_imgs = [os.path.join(mypath, f) for f in os.listdir(mypath) if f.startswith('dog')]

# Load SqueezeNet model
!git clone https://github.com/miaow1988/SqueezeNet_v1.2.git
sym, arg_params, aux_params = mx.model.load_checkpoint('/content/SqueezeNet_v1.2/model', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1, 3, 224, 224))])

# Extract the feature extractor layer
all_layers = sym.get_internals()
fe_sym = all_layers['flatten0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

# Function to get and process images
def get_image(fname):
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

# Function to extract features
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

# Extract features and train a model for different values of N
N_values = [10, 100, 500, 1000]  # Different numbers of images
for Nmax in N_values:
    # Prepare data and labels
    cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax]]
    dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax]]
    X_cvd = np.vstack([cats_features, dogs_features])
    Y_cvd = np.array(Nmax * [1] + Nmax * [0])  # 1 for cats, 0 for dogs

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, test_size=0.2, random_state=42)

    # Model training with GridSearchCV
    random_forest = RandomForestClassifier()
    param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
    grid_search = GridSearchCV(random_forest, param_grid, cv=5)
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()

    # Evaluate the model
    best_params = grid_search.best_params_
    rf = RandomForestClassifier(**best_params).fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Report results
    print(f"N: {Nmax}, Accuracy: {accuracy:.2f}, Training Time: {end_time - start_time:.2f} seconds")

Skipping, found downloaded files in "./dogs-vs-cats-redux-kernels-edition" (use force=True to force download)
fatal: destination path 'SqueezeNet_v1.2' already exists and is not an empty directory.
N: 10, Accuracy: 1.00, Training Time: 10.50 seconds
N: 100, Accuracy: 0.88, Training Time: 15.15 seconds
N: 500, Accuracy: 0.94, Training Time: 68.32 seconds
N: 1000, Accuracy: 0.94, Training Time: 168.71 seconds


# Task 2 Solution

In [6]:
# Task 2: MobileNet V2 for Dogs vs. Cats Dataset
# Import necessary libraries
!pip install mxnet==1.5.1
!pip install opendatasets
import mxnet as mx
import opendatasets as od
import os
import zipfile
import time
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

# Download and prepare the Dogs vs. Cats dataset
dataset_url = 'https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data'
od.download(dataset_url)
zip_file_path = '/content/dogs-vs-cats-redux-kernels-edition/train.zip'
extracted_dir = '/content/dogs-vs-cats-redux-kernels-edition/'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Verify the number of images (should be 12500 for each class)
mypath = os.path.join(os.getcwd(), '/content/dogs-vs-cats-redux-kernels-edition/train/')
cats_imgs = [os.path.join(mypath, f) for f in os.listdir(mypath) if f.startswith('cat')]
dogs_imgs = [os.path.join(mypath, f) for f in os.listdir(mypath) if f.startswith('dog')]
print(f"Number of cat images: {len(cats_imgs)}")
print(f"Number of dog images: {len(dogs_imgs)}")

# Load MobileNet V2 model
!git clone https://github.com/chinakook/MobileNetV2.mxnet.git
sym, arg_params, aux_params = mx.model.load_checkpoint('/content/MobileNetV2.mxnet/mbnv2', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1, 3, 224, 224))])

# Extract the feature extractor layer
all_layers = sym.get_internals()
fe_sym = all_layers['flatten1_reshape0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

# Function to get and process images
def get_image(fname):
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img is np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

# Function to extract features
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

# Prepare for comparison with SqueezeNet
squeezenet_results = {}  # This should be populated with results from Task 1
mobilenet_results = {}

# Extract features and train a model for different values of N
N_values = [10, 100, 500, 1000]  # Different numbers of images
for Nmax in N_values:
    # Prepare data and labels
    cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax]]
    dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax]]
    X_cvd = np.vstack([cats_features, dogs_features])
    Y_cvd = np.array(Nmax * [1] + Nmax * [0])  # 1 for cats, 0 for dogs

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, test_size=0.2, random_state=42)

    # Model training with GridSearchCV
    random_forest = RandomForestClassifier()
    param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
    grid_search = GridSearchCV(random_forest, param_grid, cv=5)
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()

    # Evaluate the model
    best_params = grid_search.best_params_
    rf = RandomForestClassifier(**best_params).fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Store results for comparison
    mobilenet_results[Nmax] = {
        'accuracy': accuracy,
        'training_time': end_time - start_time
    }

print("For larger datasets (like N = 5000 and 12500), you might want to consider computational resources before running the code.")
print("If you're running this in an environment like Google Colab, ensure that your session remains active during long-running processes.")

squeezenet_results = {
    10: {'accuracy': 1.00, 'training_time': 10.68},
    100: {'accuracy': 0.88, 'training_time': 14.84},
    500: {'accuracy': 0.96, 'training_time': 67.16},
    1000: {'accuracy': 0.94, 'training_time': 171.97}
}

# Comparison of SqueezeNet and MobileNet V2
for Nmax in N_values:
    print(f"Results for N={Nmax}:")
    print(f"SqueezeNet - Accuracy: {squeezenet_results[Nmax]['accuracy']}, Time: {squeezenet_results[Nmax]['training_time']}")
    print(f"MobileNet V2 - Accuracy: {mobilenet_results[Nmax]['accuracy']}, Time: {mobilenet_results[Nmax]['training_time']}")

Skipping, found downloaded files in "./dogs-vs-cats-redux-kernels-edition" (use force=True to force download)
Number of cat images: 12500
Number of dog images: 12500
fatal: destination path 'MobileNetV2.mxnet' already exists and is not an empty directory.
For larger datasets (like N = 5000 and 12500), you might want to consider computational resources before running the code.
If you're running this in an environment like Google Colab, ensure that your session remains active during long-running processes.
Results for N=10:
SqueezeNet - Accuracy: 1.0, Time: 10.68
MobileNet V2 - Accuracy: 0.75, Time: 10.764081478118896
Results for N=100:
SqueezeNet - Accuracy: 0.88, Time: 14.84
MobileNet V2 - Accuracy: 0.8, Time: 16.649003267288208
Results for N=500:
SqueezeNet - Accuracy: 0.96, Time: 67.16
MobileNet V2 - Accuracy: 0.885, Time: 84.54303693771362
Results for N=1000:
SqueezeNet - Accuracy: 0.94, Time: 171.97
MobileNet V2 - Accuracy: 0.8575, Time: 190.29948329925537


# Task 3 Solution

In [7]:
# Task 3: Predict COVID-19 from Chest X-Ray images using the best network
# Import necessary libraries
!pip install tensorflow
!pip install opendatasets
import tensorflow as tf
import opendatasets as od
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier  # Importing RandomForestClassifier
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import time

# Download the CoronaHack -Chest X-Ray-Dataset
dataset_url = 'https://www.kaggle.com/datasets/praveengovi/coronahack-chest-xraydataset'
od.download(dataset_url)

# Assuming the best network is EfficientNetB0 (replace this with your best network)
from tensorflow.keras.applications.efficientnet import EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

# Function to process and get features from images
def get_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)

    features = model.predict(img_array)
    return features.flatten()

# Define the dataset paths
dataset_path = '/content/coronahack-chest-xraydataset'
train_images_dir = os.path.join(dataset_path, 'Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train')
test_images_dir = os.path.join(dataset_path, 'Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test')
labels_file = os.path.join(dataset_path, 'Chest_xray_Corona_Metadata.csv')

# Read the CSV file for labels
df = pd.read_csv(labels_file)

# Filter out rows without image names and separate into train and test dataframes
df_train = df[df['Dataset_type'] == 'TRAIN']
df_test = df[df['Dataset_type'] == 'TEST']

# Assuming 'X_ray_image_name' column has image file names and 'Label' column has target labels
train_image_paths = [os.path.join(train_images_dir, fname) for fname in df_train['X_ray_image_name']]
test_image_paths = [os.path.join(test_images_dir, fname) for fname in df_test['X_ray_image_name']]
train_labels = df_train['Label'].values
test_labels = df_test['Label'].values

# Extract features for train and test sets
# Note: Depending on your resources, you may want to limit the number of images processed
train_features = [get_features(img_path) for img_path in train_image_paths]
test_features = [get_features(img_path) for img_path in test_image_paths]

# Define and train your classifier
clf = RandomForestClassifier()
start_time = time.time()
clf.fit(train_features, train_labels)
end_time = time.time()

# Predict and evaluate the classifier
y_pred = clf.predict(test_features)
accuracy = accuracy_score(test_labels, y_pred)

# Report the results
print(f"Model accuracy: {accuracy:.2f}")
print(f"Training time: {end_time - start_time:.2f} seconds")


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 90ms/step
Model accuracy: 0.76
Training time: 226.82 seconds


**Task 1**: SqueezeNet on Dogs vs. Cats Dataset
In Task 1, SqueezeNet shows excellent performance, achieving perfect accuracy (1.00) when trained on a smaller dataset (N=10). As the dataset size increases, there's a slight decrease in accuracy, but it remains high (0.88 to 0.96) even for N=1000. The training time increases with the dataset size, as expected, but the model remains quite efficient, with the longest training time being under three minutes (171.97 seconds) for N=1000.

**Task 2:** Comparison between SqueezeNet and MobileNet V2
The comparison between SqueezeNet and MobileNet V2 in Task 2 reveals interesting insights:

For N=10, SqueezeNet outperforms MobileNet V2 in terms of accuracy (1.0 vs. 0.75). However, the training times are comparable.
As the dataset size increases, MobileNet V2 starts to close the gap in accuracy, achieving 0.8575 for N=1000, compared to SqueezeNet's 0.94. Nevertheless, SqueezeNet consistently maintains a lead in accuracy across all dataset sizes.
In terms of training time, MobileNet V2 is slightly slower than SqueezeNet, especially noticeable as the dataset size increases.
Task 3: Using the Best Network for CoronaHack -Chest X-Ray-Dataset
In Task 3, where the goal is to predict COVID-19 from chest X-Ray images, the model achieves an accuracy of 0.76 with a training time of approximately 226.82 seconds. The specific network used for this task isn't specified in your results, but given the nature of the task, this accuracy is quite reasonable, considering the complexity and variability inherent in medical image diagnosis.

**Conclusion and Best Model**
Overall Performance: SqueezeNet appears to be the best model in terms of a balance between accuracy and training time. It consistently achieves high accuracy with relatively low training times across different dataset sizes in Task 1 and Task 2.
Efficiency and Scalability: For larger datasets (N=1000), SqueezeNet still maintains a lead in accuracy over MobileNet V2, while keeping the training time reasonable.
Task-Specific Consideration: For Task 3, depending on the network used (not specified in the results), the model's performance is respectable. If SqueezeNet was used, it would align with the conclusion that SqueezeNet is overall the best model considering the tasks and datasets.
Recommendation: Based on these results, SqueezeNet would be recommended for tasks similar to those in this series, especially when you need a good balance between accuracy and efficiency. For more specialized tasks like medical image analysis (as in Task 3), further tuning and potentially a different model might be necessary, depending on the specific requirements and data characteristics.